<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-14 13:17:19
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.28 C
-------------------
Today PnL: 94.51 K (0.75%)
Current PnL: -17.97 L (-12.74%)
CY Booked + Current PnL: -15.10 L (-10.71%)
-------------------
Total profit:  3.93 L
Total loss:  -21.90 L
-------------------
Total Booked + Current PnL: 11.88 L (10.42%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 79.85 L (62.53%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (110.84%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.58,X-MC,87.23,140775.0,25997.0,9728.0,0.25,22.65,6.91,31.13,215.0,2.67,1.14,37.13,AR,ATH,TYRES
56,PIDILITIND,2504.06,3576.00,-15.68,X-LC,45.74,93477.0,18355.0,13807.0,0.49,24.43,14.77,42.81,87.0,1.33,0.76,17.23,X40,BTT,CHEMICALS
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,84.04,147361.0,21656.0,14338.0,0.82,17.23,9.73,28.63,116.0,1.51,1.19,31.28,AR,ATH,INSURANCE
70,SBIN,760.30,863.00,-13.45,M-LC,50.00,211762.0,10282.0,16941.0,-0.33,5.10,8.00,13.51,91.0,0.61,1.71,16.01,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-21.57,X-LC,9.57,96586.0,12970.0,26107.0,-0.98,15.51,27.03,46.73,123.0,0.50,0.78,18.23,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINEORG,4771.46,7168.71,-19.14,H-SC,48.94,77188.0,-3927.0,44676.0,5.64,-4.84,57.88,50.24,128.0,-0.09,0.62,29.61,X40N,ATH,CHEMICALS
82,TITAGARH,1097.23,1548.00,-21.45,H-SC,23.40,138075.0,-53940.0,132828.0,5.33,-28.09,96.20,41.08,76.0,-0.41,1.12,16.86,XY24,NTT,ENGINEERING
75,SONACOMS,578.05,1006.00,-28.02,M-MC,89.36,93030.0,-8129.0,83020.0,4.61,-8.04,89.24,74.03,21.0,-0.10,0.75,30.74,AR,BTT,AUTO
65,SAIL,130.64,228.00,82.87,M-MC,61.17,208362.0,-16600.0,184255.0,4.31,-7.38,88.43,74.53,217.0,-0.09,1.68,20.98,XY24,BTT,STEEL
73,SIEMENS,4406.90,7969.85,-16.70,X-LC,57.45,75435.0,-34737.0,123811.0,3.45,-31.53,164.13,80.85,10.0,-0.28,0.61,11.10,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-787.38,H-SC,97.87,65520.0,-29796.0,74490.0,-3.21,-31.26,113.69,46.89,51.0,-0.40,0.53,29.23,X40N,NTT,MISC
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,62.77,100619.0,-1108.0,115480.0,-2.71,-1.09,114.77,112.43,NaN,-0.01,0.81,18.75,AR,ATH,MISC
80,TATAMOTORS,776.02,1065.00,-53.87,M-LC,81.91,183804.0,-21841.0,98427.0,-1.99,-10.62,53.55,37.24,1.0,-0.22,1.49,19.64,XY24,NTT,AUTO
7,ASIANPAINT,2961.56,4250.00,-21.82,X-LC,5.32,155366.0,-46020.0,133630.0,-1.71,-22.85,86.01,43.51,113.0,-0.34,1.26,7.16,X40,BTT,PAINTS
35,ICICIGI,1839.64,2268.71,-17.24,H-MC,25.53,136160.0,27.0,31725.0,-1.42,0.02,23.30,23.32,83.0,0.00,1.10,20.55,X40,ATH,INSURANCE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-53.87,M-LC,81.91,183804.0,-21841.0,98427.0,-1.99,-10.62,53.55,37.24,1.0,-0.22,1.49,19.64,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.41,H-LC,68.09,140875.0,7891.0,82947.0,-0.03,5.93,58.88,68.31,2.0,0.10,1.14,14.87,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-24.98,H-LC,46.81,124380.0,1377.0,61332.0,1.94,1.12,49.31,50.98,3.0,0.02,1.01,13.64,AR,ATH,REALTY
10,AWL,342.88,485.00,-68.39,H-MC,11.70,171450.0,-46279.0,136526.0,2.62,-21.26,79.63,41.45,5.0,-0.34,1.39,14.53,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-35.54,M-SC,92.55,162293.0,16063.0,126004.0,0.94,10.98,77.64,97.15,7.0,0.13,1.31,38.65,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.12,14.58,X-MC,87.23,140775.0,25997.0,9728.0,0.25,22.65,6.91,31.13,215.0,2.67,1.14,37.13,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,7.58,X-MC,14.89,189681.0,34246.0,36191.0,0.51,22.03,19.08,45.32,198.0,0.95,1.53,29.06,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.68,X-LC,45.74,93477.0,18355.0,13807.0,0.49,24.43,14.77,42.81,87.0,1.33,0.76,17.23,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,86.17,188402.0,37196.0,28468.0,0.44,24.60,15.11,43.43,37.0,1.31,1.52,30.59,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.68,M-SC,31.91,138880.0,1302.0,93105.0,2.45,0.95,67.04,68.62,55.0,0.01,1.12,20.40,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,6.49,H-MC,85.11,166670.0,26504.0,44484.0,-0.92,18.91,26.69,50.65,54.0,0.60,1.35,27.20,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,2.48,H-SC,76.60,166699.0,15335.0,30439.0,3.10,10.13,18.26,30.25,131.0,0.50,1.35,31.94,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,86.17,188402.0,37196.0,28468.0,0.44,24.60,15.11,43.43,37.0,1.31,1.52,30.59,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-5.77,H-SC,38.30,139086.0,1226.0,36538.0,2.82,0.89,26.27,27.39,74.0,0.03,1.12,23.39,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-8.80,M-LC,42.55,156085.0,5140.0,104312.0,0.07,3.41,66.83,72.51,58.0,0.05,1.26,17.52,XR,NTT,IT
18,CAMS,3643.00,5250.99,-9.33,H-SC,7.45,101942.0,-62.0,45089.0,0.40,-0.06,44.23,44.14,47.0,-0.00,0.82,16.17,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,35.11,129448.0,296.0,86769.0,1.06,0.23,67.03,67.42,71.0,0.00,1.05,25.90,XR,NTT,BANKS
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,62.77,100619.0,-1108.0,115480.0,-2.71,-1.09,114.77,112.43,NaN,-0.01,0.81,18.75,AR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-31.30,H-MC,13.83,137390.0,-2530.0,74823.0,-0.69,-1.81,54.46,51.67,19.0,-0.03,1.11,3.77,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,37.69,M-SC,80.85,119556.0,-22923.0,42442.0,0.71,-16.09,35.50,13.70,219.0,-0.54,0.97,41.11,X40N,NTT,BANKS
53,MRF,114777.60,150504.12,14.58,X-MC,87.23,140775.0,25997.0,9728.0,0.25,22.65,6.91,31.13,215.0,2.67,1.14,37.13,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-0.75,H-LC,84.04,147361.0,21656.0,14338.0,0.82,17.23,9.73,28.63,116.0,1.51,1.19,31.28,AR,ATH,INSURANCE
86,UNIONBANK,123.87,163.00,-12.58,M-LC,75.53,153154.0,12314.0,32178.0,3.01,8.74,21.01,31.59,83.0,0.38,1.24,33.04,XY24,NTT,BANKS
14,BERGEPAINT,558.54,680.00,-17.23,H-MC,59.57,177520.0,463.0,38043.0,3.15,0.26,21.43,21.75,139.0,0.01,1.44,26.98,XY24,NTT,PAINTS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TATAELXSI,7332.28,9000.00,-18.67,X-MC,100.00,85330.0,-17322.0,40668.0,1.82,-16.87,47.66,22.74,33.0,-0.43,0.69,28.26,X40N,NTT,IT
58,QUESS,711.66,986.00,NaN,NaN,98.94,80034.0,-75820.0,135898.0,0.56,-48.65,169.80,38.55,NaN,-0.56,0.65,22.55,XY24,NTT,MISC
90,VIPIND,488.80,718.00,-787.38,H-SC,97.87,65520.0,-29796.0,74490.0,-3.21,-31.26,113.69,46.89,51.0,-0.40,0.53,29.23,X40N,NTT,MISC
78,TANLA,917.30,1963.11,-40.21,H-SC,96.81,146434.0,-92981.0,365939.0,0.66,-38.84,249.90,114.01,29.0,-0.25,1.18,34.30,AR,ATH,IT
1,AARTIIND,487.04,919.00,35.42,M-SC,95.74,77755.0,-3581.0,75718.0,-0.24,-4.40,97.38,88.69,118.0,-0.05,0.63,31.12,XR,NTT,CHEMICALS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,40.98
2,50,69.51


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.91
X40     14.46
X40N    14.04
AR      13.77
XR      10.31
XY25    10.18
X200     1.22
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.37
X-LC    18.21
M-SC    16.63
M-LC     8.37
H-MC     7.86
X-MC     6.77
H-LC     5.66
M-MC     5.56
X-SC     2.79
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.74,-2.17,34.67
BANKS,11.14,-9.33,62.95
IT,9.31,-20.16,87.20
FINANCE,7.46,-19.85,73.94
MISC,7.17,-27.65,82.58
INSURANCE,6.06,-3.00,34.05
PAINTS,5.59,-19.13,54.39
ELECTRICAL,4.13,-9.04,64.60
HEALTHCARE,3.64,-3.35,36.72


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3035054.0,27
XR,1268975.0,15
AR,1250930.0,15
X40N,1196603.0,17
X40,805466.0,12
SR,209035.0,2
XY25,151984.0,5
X200,66696.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2418243.0,22
M-SC,1949362.0,20
X-LC,732185.0,12
M-MC,579843.0,5
H-MC,425367.0,7
X-SC,363736.0,4
M-LC,344353.0,6
X-MC,318875.0,6
L-SC,317770.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922962.0      6
M-SC       XY24      767050.0      6
H-SC       X40N      552959.0      7
           AR        543196.0      3
M-SC       XR        430113.0      5
M-MC       XY24      410054.0      3
X-LC       X40       376363.0      5
M-SC       X40N      372682.0      5
H-SC       XR        303280.0      4
M-LC       XY24      223100.0      4
M-SC       X40       199396.0      2
H-LC       AR        191119.0      4
X-LC       AR        189379.0      3
H-MC       XY24      174569.0      2
L-SC       XR        164079.0      2
X-MC       XY24      140427.0      2
X-SC       SR        123186.0      1
           XR        122030.0      1
           X40N      118520.0      2
X-MC       X40N      107958.0      2
M-LC       XR        104312.0      1
X-LC       XY25       99747.0      3
H-SC       X40        95846.0      2
M-SC       AR         94272.0      1
L-SC       XY24       88298.0      1
M-MC       XR         86769.0      1
M-SC       SR         85849.0      1
M-MC       AR         83020.0      1
H-MC       AR         74823.0      1
           X40        73099.0      2
X-LC       X200       66696.0      1
L-SC       AR         65393.0      1
X-MC       X40        60762.0      1
H-MC       XR         58392.0      1
           X40N       44484.0      1
H-LC       XY25       35296.0      1
M-LC       XY25       16941.0      1
X-MC       AR          9728.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
